In [5]:
import joblib
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Carregar os modelos salvos
vectorizer = joblib.load('tfidf_vectorizer_for_recommendation.joblib')
svd = joblib.load('svd_model_for_recommendation.joblib')
X_svd_transformed = joblib.load('svd_transformed_data_for_recommendation.joblib')

# Carregar o dataset para as recomendações
df = pd.read_excel('ready_for_filtering.xlsx')
df.fillna('', inplace=True)

# Função para recomendar métricas para um novo usuário
def recommend_metrics_for_new_user(new_user_data, vectorizer, svd, X_svd_transformed, df, top_n=5):
    # Vetorizar o novo perfil usando o mesmo TF-IDF vectorizer
    new_user_tfidf = vectorizer.transform([new_user_data])
    
    # Reduzir dimensionalidade com SVD
    new_user_svd = svd.transform(new_user_tfidf)
    
    # Calcular a similaridade entre o novo perfil e todos os perfis existentes
    cosine_sim_new_user = cosine_similarity(new_user_svd, X_svd_transformed).flatten()
    
    # Ordenar por similaridade (maior -> menor)
    sim_scores = list(enumerate(cosine_sim_new_user))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Selecionar os top_n perfis mais semelhantes
    sim_scores = sim_scores[:top_n]
    
    # Mostrar a similaridade dos perfis recomendados
    print(f"\nPerfis mais semelhantes ao novo usuário:")
    for i, score in sim_scores:
        print(f"Perfil {i}: Similaridade = {score*100:.2f}% - Métrica recomendada: {df['sanitized_metrics'].iloc[i]}")
    
    # Recomendar as métricas dos perfis mais semelhantes
    recommended_metrics = [df['sanitized_metrics'].iloc[i] for i, _ in sim_scores]
    
    return recommended_metrics

user_profiles = [
  
   # 'teamleader maisde20 grandeempresa scrumscrumbankanban sim sim não não sim produtoprocesso'
    'productmanager0a5grandeempresascrumscrumbansimsimnãonãonãocronogramaeprogressopessoas'
]

# Simular métricas recomendadas para cada perfil de usuário
for user_data in user_profiles:
    recommended_metrics = recommend_metrics_for_new_user(user_data, vectorizer, svd, X_svd_transformed, df, top_n=5)
    print(f"\nMétricas recomendadas para o usuário ({user_data}): {recommended_metrics}")



Perfis mais semelhantes ao novo usuário:
Perfil 0: Similaridade = 0.00% - Métrica recomendada: Ideal team capacity
Perfil 1: Similaridade = 0.00% - Métrica recomendada: NPS
Perfil 2: Similaridade = 0.00% - Métrica recomendada: Burndown
Perfil 3: Similaridade = 0.00% - Métrica recomendada: Outstanding bugs
Perfil 4: Similaridade = 0.00% - Métrica recomendada: Throughput

Métricas recomendadas para o usuário (productmanager0a5grandeempresascrumscrumbansimsimnãonãonãocronogramaeprogressopessoas): ['Ideal team capacity', 'NPS', 'Burndown', 'Outstanding bugs', 'Throughput']
